In [2]:
# -*- coding: utf-8 -*-
"""
Pipeline này triển khai Feature Engineering nâng cao:
1. Giữ lại phương pháp dự đoán Age.
2. Phân loại (Binning) cho Age và Fare.
3. Tạo các đặc trưng tương tác (Interaction Features).
"""

import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# =============================================================================
# 1. TẢI DỮ LIỆU GỐC
# =============================================================================
try:
    train_df_raw = pd.read_csv('train.csv')
    test_df_raw = pd.read_csv('test.csv')
    print("Đã tải dữ liệu thô thành công.")
except FileNotFoundError:
    print("Lỗi: Không tìm thấy file train.csv hoặc test.csv.")
    exit()

# =============================================================================
# 2. HÀM XỬ LÝ DỮ LIỆU NÂNG CAO
# =============================================================================
def apply_advanced_feature_engineering(train_df, test_df):
    
    y_target = train_df['Survived']
    combined_df = pd.concat([train_df.drop('Survived', axis=1), test_df], ignore_index=True)
    
    # --- Bước A: Điền các giá trị thiếu đơn giản ---
    combined_df['Embarked'].fillna(combined_df['Embarked'].mode()[0], inplace=True)
    combined_df['Fare'].fillna(combined_df['Fare'].median(), inplace=True)
    
    # --- Bước B: Dự đoán 'Age' (giữ nguyên logic hiệu quả) ---
    # (Code dự đoán Age giữ nguyên như cũ)
    df_for_age_pred = combined_df.copy()
    df_for_age_pred['Title'] = df_for_age_pred['Name'].str.extract(' ([A-Za-z]+)\\.', expand=False)
    df_for_age_pred['Title'] = df_for_age_pred['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare').replace({'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs'})
    df_for_age_pred['Sex'] = df_for_age_pred['Sex'].map({'male': 0, 'female': 1})
    df_for_age_pred['Embarked'] = df_for_age_pred['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
    df_for_age_pred['Title'] = df_for_age_pred['Title'].map({"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}).fillna(0)
    age_features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Title']
    age_regressor = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
    age_regressor.fit(df_for_age_pred[df_for_age_pred['Age'].notna()][age_features], df_for_age_pred[df_for_age_pred['Age'].notna()]['Age'])
    predicted_age = age_regressor.predict(df_for_age_pred[df_for_age_pred['Age'].isna()][age_features])
    combined_df.loc[combined_df['Age'].isna(), 'Age'] = predicted_age
    print("Đã dự đoán và điền các giá trị 'Age' bị thiếu.")

    # --- Bước C: FEATURE ENGINEERING NÂNG CAO ---
    print("Đang thực hiện Feature Engineering nâng cao...")
    
    # 1. Tạo các đặc trưng cơ bản
    combined_df['FamilySize'] = combined_df['SibSp'] + combined_df['Parch'] + 1
    combined_df['IsAlone'] = (combined_df['FamilySize'] == 1).astype(int)
    
    # 2. Phân loại (Binning) cho 'Age' và 'Fare'
    # Dùng qcut cho Fare vì phân bố của nó bị lệch, qcut chia đều số lượng mẫu vào các bin
    combined_df['FareBin'] = pd.qcut(combined_df['Fare'], 4, labels=['Very_Low', 'Low', 'High', 'Very_High'])
    combined_df['AgeBin'] = pd.cut(combined_df['Age'].astype(int), bins=[0, 12, 20, 40, 60, 120], labels=['Child', 'Teenager', 'Adult', 'Middle_Age', 'Senior'])

    # 3. Tạo các đặc trưng tương tác
    combined_df['Age*Class'] = combined_df['Age'] * combined_df['Pclass']
    # Thêm 1 để tránh chia cho 0
    combined_df['FarePerPerson'] = combined_df['Fare'] / (combined_df['FamilySize'] + 1e-6)

    # 4. Trích xuất thông tin từ 'Title' và 'Cabin'
    combined_df['Title'] = combined_df['Name'].str.extract(' ([A-Za-z]+)\\.', expand=False)
    combined_df['Title'] = combined_df['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare').replace({'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs'})
    combined_df['Deck'] = combined_df['Cabin'].str[0].fillna('U')
    
    print("   -> Đã tạo các đặc trưng: FamilySize, IsAlone, FareBin, AgeBin, Age*Class, FarePerPerson, Title, Deck.")

    # --- Bước D: Mã hóa và hoàn thiện ---
    combined_s = combined_df.drop(columns=['Name', 'Ticket', 'Cabin', 'SibSp', 'Parch'])
    
    # One-Hot Encode tất cả các cột categorical
    categorical_cols = ['Sex', 'Embarked', 'Title', 'Deck', 'FareBin', 'AgeBin']
    combined_final = pd.get_dummies(combined_s, columns=categorical_cols, drop_first=True, dtype=int)
    
    print("Đã hoàn thành mã hóa One-Hot.")

    # Tách lại thành train và test set
    train_final = combined_final.iloc[:len(train_df)]
    test_final = combined_final.iloc[len(train_df):]
    train_final['Survived'] = y_target

    return train_final, test_final

# =============================================================================
# 3. THỰC THI VÀ LƯU KẾT QUẢ
# =============================================================================
train_final_df, test_final_df = apply_advanced_feature_engineering(train_df_raw, test_df_raw)

save_dir = "Data_clean_v3"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Lưu các file đã xử lý
train_final_df.to_csv(f'{save_dir}/train_cleaned.csv', index=False)
test_final_df.to_csv(f'{save_dir}/test_cleaned.csv', index=False)

print(f"\nĐã lưu các file đã xử lý vào thư mục '{save_dir}'.")
print("\nXem 5 dòng đầu của dữ liệu huấn luyện cuối cùng:")
print(train_final_df.head())
print(f"\nSố lượng đặc trưng cuối cùng: {len(train_final_df.columns) - 2}")

Đã tải dữ liệu thô thành công.


C:\Users\DELL\AppData\Local\Temp\ipykernel_8516\1073593138.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_df['Embarked'].fillna(combined_df['Embarked'].mode()[0], inplace=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_8516\1073593138.py:36: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alway

Đã dự đoán và điền các giá trị 'Age' bị thiếu.
Đang thực hiện Feature Engineering nâng cao...
   -> Đã tạo các đặc trưng: FamilySize, IsAlone, FareBin, AgeBin, Age*Class, FarePerPerson, Title, Deck.
Đã hoàn thành mã hóa One-Hot.

Đã lưu các file đã xử lý vào thư mục 'Data_clean_v3'.

Xem 5 dòng đầu của dữ liệu huấn luyện cuối cùng:
   PassengerId  Pclass   Age     Fare  FamilySize  IsAlone  Age*Class  \
0            1       3  22.0   7.2500           2        0       66.0   
1            2       1  38.0  71.2833           2        0       38.0   
2            3       3  26.0   7.9250           1        1       78.0   
3            4       1  35.0  53.1000           2        0       35.0   
4            5       3  35.0   8.0500           1        1      105.0   

   FarePerPerson  Sex_male  Embarked_Q  ...  Deck_T  Deck_U  FareBin_Low  \
0       3.624998         1           0  ...       0       1            0   
1      35.641632         0           0  ...       0       0            0   

C:\Users\DELL\AppData\Local\Temp\ipykernel_8516\1073593138.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_final['Survived'] = y_target
